In [1]:
# Getting the data from the API
import requests

def getCitibikeData():
    # Let's get the data from the Citibike API
    url = 'http://www.citibikenyc.com/stations/json'
    results = requests.get(url).json() 

    # We only need a subset of the data in the JSON returned by the Citibike API, so we keep only we need
    data = results["stationBeanList"]
    return data



In [2]:
import MySQLdb as mdb
import sys


con = mdb.connect(host = 'localhost', 
                  user = 'root', 
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True)


def createDatabase():
    # Create the Citibike database
    db_name = 'citibike2'
    create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

    # Create a database
    cursor = con.cursor()
    cursor.execute(create_db_query)
    cursor.close()

In [3]:
# Create a table with time-invariant characteristics of the Docks
def createTimeInvariantTable():
    cursor = con.cursor()
    db_name = 'citibike2'
    table_name = 'Docks'

    create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                    (station_id int, 
                                    station_name varchar(250), 
                                    total_docks int,
                                    latitude float,
                                    longitude float,
                                    PRIMARY KEY(station_id)
                                    )'''.format(db=db_name, table=table_name)
    cursor.execute(create_table_query)
    cursor.close()

In [4]:
# Create a table with time-varying characteristics of the Citibike stations

def createTimeVaryingTable():
    cursor = con.cursor()
    db_name = 'citibike2'
    table_name = 'Docks_Status'

    create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                    (station_id int,
                                    last_communication_time datetime,
                                    available_bikes int, 
                                    available_docks int,
                                    status_key int,
                                    status_value varchar(50),
                                    test_station bool,
                                    PRIMARY KEY(station_id, last_communication_time),
                                    FOREIGN KEY(station_id) 
                                            REFERENCES {db}.Docks(station_id)
                                    )'''.format(db=db_name, table=table_name)
    cursor.execute(create_table_query)
    cursor.close()

In [5]:
from datetime import date, datetime, timedelta


def insertTimeInvariantData():
    query_template = '''INSERT IGNORE INTO 
        citibike2.Docks(station_id,station_name, total_docks, latitude, longitude) 
        VALUES (%s, %s, %s, %s, %s)'''

    cursor = con.cursor()

    for entry in data:
        dockid = entry["id"]
        addr = entry["stationName"]
        docks = entry["totalDocks"]
        lon = entry["longitude"]
        lat = entry["latitude"]
        print("Inserting station", dockid, "at", addr)
        query_parameters = (dockid, addr, docks, lat,lon)
        cursor.execute(query_template, query_parameters)

    con.commit()
    cursor.close()

In [6]:
def insertTimeVaryingData():
    query_template = '''INSERT IGNORE INTO 
        citibike2.Docks_Status(
        station_id, last_communication_time,
         available_bikes, available_docks,
                                    status_key,
                                    status_value,
                                    test_station) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)'''

    cursor = con.cursor()

    
    for entry in data:
        dockid = entry["id"]
        bikes = entry["availableBikes"]
        docks = entry["availableDocks"]
        status_time = entry["lastCommunicationTime"]
        status_key = entry["statusKey"]
        status_value = entry["statusValue"]
        test = entry["testStation"]
        # The API returns a string like "2017-01-30 07:18:12 PM"
        date = datetime.strptime(entry["lastCommunicationTime"], 
                             '%Y-%m-%d %I:%M:%S %p')
        
        print("Inserting station", dockid, "at", date)
        query_parameters = (dockid, date, bikes, docks, status_key, status_value, test)
        cursor.execute(query_template, query_parameters)

    con.commit()
    cursor.close()

In [7]:
data = getCitibikeData()

createDatabase()

createTimeInvariantTable()
insertTimeInvariantData()

createTimeVaryingTable()
insertTimeVaryingData()

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:18: Warning: (1007, "Can't create database 'citibike2'; database exists")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:15: Warning: (1050, "Table 'Docks' already exists")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '72' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '79' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '82' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '83' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '116' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '119' for key 'PRIMARY'")
/usr/local/l

Inserting station 72 at W 52 St & 11 Ave
Inserting station 79 at Franklin St & W Broadway
Inserting station 82 at St James Pl & Pearl St
Inserting station 83 at Atlantic Ave & Fort Greene Pl
Inserting station 116 at W 17 St & 8 Ave
Inserting station 119 at Park Ave & St Edwards St
Inserting station 120 at Lexington Ave & Classon Ave
Inserting station 127 at Barrow St & Hudson St
Inserting station 128 at MacDougal St & Prince St
Inserting station 137 at E 56 St & Madison Ave
Inserting station 143 at Clinton St & Joralemon St
Inserting station 144 at Nassau St & Navy St
Inserting station 146 at Hudson St & Reade St
Inserting station 147 at Greenwich St & Warren St
Inserting station 150 at E 2 St & Avenue C
Inserting station 151 at Cleveland Pl & Spring St
Inserting station 152 at Warren St & Church St
Inserting station 153 at E 40 St & 5 Ave
Inserting station 157 at Henry St & Atlantic Ave
Inserting station 161 at LaGuardia Pl & W 3 St
Inserting station 164 at E 47 St & 2 Ave
Inserting s

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '509' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '511' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '513' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '514' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '515' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '516' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '517' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '518' for key 'PRIMARY'")
/usr/loc

Inserting station 511 at E 14 St & Avenue B
Inserting station 513 at W 56 St & 10 Ave
Inserting station 514 at 12 Ave & W 40 St
Inserting station 515 at W 43 St & 10 Ave
Inserting station 516 at E 47 St & 1 Ave
Inserting station 517 at Pershing Square South
Inserting station 518 at E 39 St & 2 Ave
Inserting station 519 at Pershing Square North
Inserting station 520 at W 52 St & 5 Ave
Inserting station 522 at E 51 St & Lexington Ave
Inserting station 523 at W 38 St & 8 Ave
Inserting station 524 at W 43 St & 6 Ave
Inserting station 525 at W 34 St & 11 Ave
Inserting station 526 at E 33 St & 5 Ave
Inserting station 527 at E 33 St & 2 Ave
Inserting station 528 at 2 Ave & E 31 St
Inserting station 529 at W 42 St & 8 Ave
Inserting station 530 at 11 Ave & W 59 St
Inserting station 531 at Forsyth St & Broome St
Inserting station 532 at S 5 Pl & S 4 St
Inserting station 533 at Broadway & W 39 St
Inserting station 534 at Water - Whitehall Plaza
Inserting station 536 at 1 Ave & E 30 St
Inserting s

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3305' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3306' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3307' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3308' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3309' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3310' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3311' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: Warning: (1062, "Duplicate entry '3312' for key 'PRIMARY'")


...

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '359-2017-01-30 19:33:44' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '363-2017-01-30 19:31:14' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '364-2017-01-30 19:33:21' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '365-2017-01-30 19:31:52' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '366-2017-01-30 19:31:13' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '367-2017-01-26 09:23:44' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '368-2017-01-30 19:33:08' for key 'PRI

 364 at 2017-01-30 19:33:21
Inserting station 365 at 2017-01-30 19:31:52
Inserting station 366 at 2017-01-30 19:31:13
Inserting station 367 at 2017-01-26 09:23:44
Inserting station 368 at 2017-01-30 19:33:08
Inserting station 369 at 2017-01-30 19:32:39
Inserting station 372 at 2017-01-30 19:33:09
Inserting station 373 at 2017-01-30 19:31:08
Inserting station 376 at 2017-01-30 19:33:18
Inserting station 377 at 2017-01-30 19:32:44
Inserting station 379 at 2017-01-30 19:32:42
Inserting station 380 at 2017-01-30 19:32:51
Inserting station 382 at 2017-01-30 19:33:50
Inserting station 383 at 2017-01-30 19:31:28
Inserting station 384 at 2017-01-30 19:30:41
Inserting station 385 at 2017-01-30 19:34:03
Inserting station 386 at 2017-01-30 19:30:41
Inserting station 387 at 2017-01-30 19:33:25
Inserting station 388 at 2017-01-30 19:30:44
Inserting station 389 at 2017-01-30 19:33:36
Inserting station 390 at 2017-01-30 19:34:13
Inserting station 391 at 2017-01-30 19:32:45
Inserting station 392 at 20

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3126-2017-01-30 19:33:01' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3127-2017-01-30 19:33:44' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3128-2017-01-30 19:32:51' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3129-2017-01-30 19:32:43' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3131-2017-01-30 19:31:57' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3134-2017-01-30 19:32:37' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3135-2017-01-30 19:31:50' for k

 at 2017-01-30 19:32:51
Inserting station 3129 at 2017-01-30 19:32:43
Inserting station 3131 at 2017-01-30 19:31:57
Inserting station 3132 at 2017-01-30 19:34:05
Inserting station 3134 at 2017-01-30 19:32:37
Inserting station 3135 at 2017-01-30 19:31:50
Inserting station 3136 at 2017-01-30 19:33:21
Inserting station 3137 at 2017-01-30 19:34:28
Inserting station 3139 at 2017-01-30 19:33:11
Inserting station 3140 at 2017-01-30 19:33:47
Inserting station 3141 at 2017-01-30 19:33:20
Inserting station 3142 at 2017-01-30 19:34:22
Inserting station 3143 at 2017-01-30 19:33:21
Inserting station 3144 at 2017-01-30 19:32:34
Inserting station 3145 at 2017-01-30 19:33:47
Inserting station 3146 at 2017-01-30 19:31:36
Inserting station 3147 at 2017-01-30 19:34:31
Inserting station 3148 at 2017-01-30 19:34:17
Inserting station 3150 at 2017-01-30 19:33:04
Inserting station 3151 at 2017-01-30 19:33:25
Inserting station 3152 at 2017-01-30 19:32:07
Inserting station 3153 at 2016-10-25 10:55:55
Inserting 

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3423-2017-01-30 19:32:31' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3424-2017-01-30 19:32:30' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3425-2017-01-30 19:30:47' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3428-2017-01-30 19:33:32' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3429-2017-01-30 19:33:47' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3430-2017-01-30 19:32:36' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:28: Warning: (1062, "Duplicate entry '3434-2017-01-30 19:30:49' for k